In [12]:
import numpy as np 
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import itertools as it
from tqdm import tqdm

In [33]:
#DATA CLEANING CELL
crime_data = pd.read_csv('Crime_Data_from_2020_to_Present.csv')
# print(len(my_data.columns))
crime_data = crime_data.drop(columns=['Vict Descent','Status','Status Desc'])
# print(len(my_data.columns))
all_c = list(crime_data.columns)
final = []
for i in all_c:
    if 'Crm' not in i:
        final.append(i)
final.append('Crm Cd 1')
print(final)
crime_data.dropna(inplace=True,subset=final)
crime_data = crime_data[crime_data['LON'] != 0]
crime_data = crime_data[crime_data['LAT'] != 0]

print(len(crime_data))
# CODE TO DROP CRIMES WITH ONLY A FEW INCIDENTS
X = crime_data.drop(columns=['Crm Cd 1', 'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 'Crm Cd'])
y = crime_data['Crm Cd']
# allowed number of crime incidents to keep in data
num_incidents = 100
crimes = []
for i in set(y):
    if len(crime_data[crime_data['Crm Cd']==i])>num_incidents:
        crimes.append(i)

new_df = crime_data[crime_data['Crm Cd'].isin(crimes)]
new_df = pd.get_dummies(new_df, drop_first=True, columns=["Vict Sex"],dtype = int)
new_df['Vict Sex'] = new_df['Vict Sex_H'] + 2*new_df['Vict Sex_M'] + 3*new_df['Vict Sex_X']


['DR_NO', 'Date Rptd', 'DATE OCC', 'TIME OCC', 'AREA', 'AREA NAME', 'Rpt Dist No', 'Part 1-2', 'Mocodes', 'Vict Age', 'Vict Sex', 'Premis Cd', 'Premis Desc', 'Weapon Used Cd', 'Weapon Desc', 'LOCATION', 'Cross Street', 'LAT', 'LON', 'Crm Cd 1']
67176


In [34]:
list(new_df.columns.values)

['DR_NO',
 'Date Rptd',
 'DATE OCC',
 'TIME OCC',
 'AREA',
 'AREA NAME',
 'Rpt Dist No',
 'Part 1-2',
 'Crm Cd',
 'Crm Cd Desc',
 'Mocodes',
 'Vict Age',
 'Premis Cd',
 'Premis Desc',
 'Weapon Used Cd',
 'Weapon Desc',
 'Crm Cd 1',
 'Crm Cd 2',
 'Crm Cd 3',
 'Crm Cd 4',
 'LOCATION',
 'Cross Street',
 'LAT',
 'LON',
 'Vict Sex_H',
 'Vict Sex_M',
 'Vict Sex_X',
 'Vict Sex']

In [40]:
features = ['TIME OCC','AREA','Rpt Dist No','Vict Age','Vict Sex','Weapon Used Cd','LAT','LON']
labels = new_df['Crm Cd 1']
X, X_test, y, y_test = train_test_split(new_df[features], labels,random_state=2)

In [53]:
def find_best(criterion="aic"):
    features = ['TIME OCC','AREA','Rpt Dist No','Vict Age','Vict Sex','Weapon Used Cd','LAT','LON']
    labels = new_df['Crm Cd 1']
    X, X_test, y, y_test = train_test_split(new_df[features], labels,random_state=2)
    train = X
    X1 = train[features].values
    results = GradientBoostingClassifier(max_features=15).fit(X1,y)
    crit = results.score(X_test[features],y_test)
    best_score = crit
    best_f = features

    for k in tqdm(range(1,len(features))):
        for combo in it.combinations(features,k):
            # print(combo)
            # print(train[list(combo)])
            X1 = train[list(combo)].values
            results = GradientBoostingClassifier(max_features=15).fit(X1,y)
            crit = results.score(X_test[list(combo)],y_test)
            if crit > best_score:
                best_score = crit
                best_f = combo
    return best_score,best_f
    # print(f"best: {best[-1]}")

In [54]:
best_score, best_f = find_best()

/Users/mckayladavis/opt/anaconda3/envs/acme/lib/python3.10/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(
  0%|          | 0/7 [00:00<?, ?it/s]/Users/mckayladavis/opt/anaconda3/envs/acme/lib/python3.10/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(
/Users/mckayladavis/opt/anaconda3/envs/acme/lib/python3.10/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(
/Users/mckayladavis/opt/anaconda3/envs/acme/lib/python3.10/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(
/Users/mckayladavis/opt/anaconda3/envs/acme/lib/python3.10/site-packages/sklearn/base.py:457: UserWarning: X has feature names,

KeyboardInterrupt: 

In [ ]:
best_f = features

In [ ]:
X, X_test, y, y_test = train_test_split(new_df[features], labels,random_state=2)
X = X[best_f]
X_test = X_test[best_f]

loss = ['log_loss']
n_estimators = [50,100]
max_depth = [1,9]
min_samples_leaf = [1,4,8]

max_loss = 0
max_n = 0
max_d = 0
max_leaf = 0
best_score = 0

for l in loss:
  for n in n_estimators:
    for md in max_depth:
      for ml in min_samples_leaf:
        model = GradientBoostingClassifier(loss = l, n_estimators = n, max_depth = md, min_samples_leaf = ml,max_features=10)
        model.fit(X,y)
        score = model.score(X_test,y_test)
        if score > best_score:
          max_loss = l
          max_n = n
          max_d = md
          max_leaf = ml
          best_score = score
print("Best Loss:", max_loss)
print("Best n_estimators:", max_n)
print("Best max_d:", md)
print("Best min_samples_leaf:", max_leaf)
print("Best Score:", best_score)

In [ ]:
loss = ['quantile', 'huber', 'squared_error', 'absolute_error']
n_estimators = [50,100]
max_depth = [1,9]
min_samples_leaf = [1,4,8]

max_loss = 0
max_n = 0
max_d = 0
max_leaf = 0
best_score = 0

for l in loss:
  for n in n_estimators:
    for md in max_depth:
      for ml in min_samples_leaf:
        model = GradientBoostingRegressor(loss = l, n_estimators = n, max_depth = md, min_samples_leaf = ml,max_features=10)
        model.fit(X,y)
        score = model.score(X_test,y_test)
        if score > best_score:
          max_loss = l
          max_n = n
          max_d = md
          max_leaf = ml
          best_score = score
print("Best Loss:", max_loss)
print("Best n_estimators:", max_n)
print("Best max_d:", md)
print("Best min_samples_leaf:", max_leaf)
print("Best Score:", best_score)

In [ ]:
knn = xgb.XGBClassifier(objective = 'multi:softmax')
# Specify values for certain hyperparameters
param_grid = {"alpha": [1,3,5],
                 "gamma": [1,3,5],
                 "lambda": [3,5,7],
                 "eta": [.3,.5,.9]}
knn_gs = GridSearchCV(knn, param_grid,cv=2)
# Run the actual search. This may take some time.
knn_gs.fit(X, y)
# After fitting, you can access data about the results.
print('best paramaters: ' + str(knn_gs.best_params_), 'best score: ' +str(knn_gs.best_score_), sep='\n')